In [10]:
import requests
import json
import os
from pathlib import Path
import editdistance
import pickle

In [2]:
tokens = json.loads(requests.post(
    'https://openai-proxy.tcsbank.ru/auth/v1/token',
    json={
        'username': f"{os.getenv("GREENPLUM_USER")}@tcsbank.ru",
        'password': os.getenv("GREENPLUM_PASSWORD")
    }).text
)

In [3]:
headers = {
    "Authorization": f"Bearer {tokens['access_token']}",  # Укажите реальный токен
    "Content-Type": "application/json"
}

In [4]:
train_names = ['11', '15', '17', '17об', '18об', '19', '19об', '20', '20об', '21', '21об', '22', '23', '23об', '24', '24об', '25', '25об', '26', '26об', '27', '27об', '28', '28об', '29', '29об', '2об', '30об', '31', '31об', '32', '32об', '33', '33об', '34', '34об', '35', '35об', '36', '38', '38об', '39', '39об', '40', '40об', '41', '41об', '42', '42об', '43', '43об', '44', '45об', '46', '46об', '47', '47об', '48', '48об', '49', '49об', '50', '50об', '51', '51об', '52', '52об', '53', '53об', '54', '54об', '55', '55об', '56', '56об', '57', '57об', '58']
val_names = ['58об', '59', '59об', '60', '60об']
test_names = ['61', '75', '75об', '7об', '95']

# train_names = ['11', '15', '17', '17об', '18', '18об', '19', '19об', '20', '20об', '21', '21об', '22', '22об', '23', '23об', '24', '24об', '25', '25об', '26', '26об', '27', '27об', '28', '28об', '29', '29об', '2об', '30', '30об', '31', '31об', '32', '32об', '33', '33об', '34', '34об', '35', '35об', '36', '38', '38об', '39', '39об', '40', '40об', '41', '41об', '42', '42об', '43', '43об', '44', '45об', '46', '46об', '47', '47об', '48', '48об', '49', '49об', '50', '50об', '51', '51об', '52', '52об', '53', '53об', '54', '54об', '55', '55об', '56', '56об', '57', '57об', '58', '58об']
# val_names = ['59', '59об', '60', '60об']
# #test_names = ['61', '75', '75об', '7об', '88об', '95']
# test_names = ['61', '75', '75об', '7об', '95']

In [5]:
def get_human_and_model_pages(names, texts_path="../../Подстрочник/Дневник_С-К/438-1-219", prefix="438-1-219 л", skip_hashtag=True):
    texts_path = Path(texts_path)
    human_pages = []
    model_pages = []

    for name in names:
        human_lines_filtered = []
        model_lines_filtered = []
        human_path = texts_path / "Human_text" / f"{prefix}{name}.txt"
        model_path = texts_path / "Model_text" / f"{prefix}{name}.txt"
        assert human_path.is_file()
        assert model_path.is_file()
        with open(human_path, "r", encoding="utf-8-sig") as human_file:
            human_lines = human_file.readlines()
        with open(model_path, "r", encoding="utf-8-sig") as model_file:
            model_lines = model_file.readlines()

        # Убираем пустую строку в конце
        if human_lines[-1].strip() == "":
            human_lines = human_lines[:-1]
        if model_lines[-1].strip() == "":
            model_lines = model_lines[:-1]

        if len(human_lines) != len(model_lines) or len(model_lines) == 0:
            print(f"len(human_lines) != len(model_lines) or len(model_lines) == 0 for {name}")
            continue

        for human_line, model_line in zip(human_lines, model_lines):
            human_line = human_line.replace("\u200b", "").strip()
            model_line = model_line.replace("\u200b", "").strip()
            if skip_hashtag and "#" in human_line:
                continue
            if human_line[-1] == "$":
                human_line = human_line[:-1]
            if model_line[-1] == "$":
                model_line = model_line[:-1]
            human_lines_filtered.append(human_line)
            model_lines_filtered.append(model_line)

        human_pages.append("\n".join(human_lines_filtered))
        model_pages.append("\n".join(model_lines_filtered))

    return human_pages, model_pages

In [6]:
def calc_cer_from_pages(human_pages, model_pages, by_lines=True):
    cer_sum = 0
    n_chars_gt = 0
    for human_page, model_page in zip(human_pages, model_pages, strict=True):
        human_page = human_page.strip()
        model_page = model_page.strip()
        if by_lines:
            for human_line, model_line in zip(human_page.split("\n"), model_page.split("\n"), strict=True):
                cer_sum += editdistance.eval(human_line, model_line)
                n_chars_gt += len(human_line)
        else:
            human_page = human_page.replace("\n", " ")
            model_page = model_page.replace("\n", " ")
            cer_sum += editdistance.eval(human_page, model_page)
            n_chars_gt += len(human_page)
    return cer_sum / n_chars_gt

In [7]:
import re


def format_string_for_wer(str):
    str = re.sub('([\[\]{}/\\()\"\'&+*=<>?.;:,!\-—_€#%°])', r' \1 ', str)
    str = re.sub('([ \n])+', " ", str).strip()
    return str


def edit_wer_from_list(truth, pred):
    edit = 0
    for pred, gt in zip(pred, truth):
        gt = format_string_for_wer(gt)
        pred = format_string_for_wer(pred)
        gt = gt.split(" ")
        pred = pred.split(" ")
        edit += editdistance.eval(gt, pred)
    return edit


def nb_words_from_list(list_gt):
    len_ = 0
    for gt in list_gt:
        gt = format_string_for_wer(gt)
        gt = gt.split(" ")
        len_ += len(gt)
    return len_

def wer_from_list_str(str_gt, str_pred):
    len_ = 0
    edit = 0
    for pred, gt in zip(str_pred, str_gt):
        gt = format_string_for_wer(gt)
        pred = format_string_for_wer(pred)
        gt = gt.split(" ")
        pred = pred.split(" ")
        edit += editdistance.eval(gt, pred)
        len_ += len(gt)
    cer = edit / len_
    return cer

<>:5: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\['
/var/folders/pp/0c5gvmy13ddcb_0yx8m9lfkw0000gq/T/ipykernel_37444/534092124.py:5: SyntaxWarning: invalid escape sequence '\['
  str = re.sub('([\[\]{}/\\()\"\'&+*=<>?.;:,!\-—_€#%°])', r' \1 ', str)


In [8]:
def calc_wer_from_pages(human_pages, model_pages):
    human_lines = []
    model_lines = []
    for human_page, model_page in zip(human_pages, model_pages, strict=True):
        human_page = human_page.strip()
        model_page = model_page.strip()
        for human_line, model_line in zip(human_page.split("\n"), model_page.split("\n"), strict=True):
            human_lines.append(human_line)
            model_lines.append(model_line)
    return wer_from_list_str(human_lines, model_lines)

In [ ]:
human_pages_val, model_pages_val = get_human_and_model_pages(val_names)
human_pages_test, model_pages_test = get_human_and_model_pages(test_names)

In [ ]:
calc_cer_from_pages(human_pages_val, model_pages_val)

0.28349584687612855

In [ ]:
calc_cer_from_pages(human_pages_test, model_pages_test)

0.22167200699097

In [10]:
def make_system_prompt(human_pages, model_pages):
    prompt = f"""Твоя задача - корректировать входной текст, исправляя в нем ошибки.
Это текст, распознанный моделью компьютерного зрения с рукописей. Рукописи написаны на русском языке 19 века (также иногда присутствуют другие языки).
Цель - получить максимально близкий к рукописи вариант расшифровки.
Модель допускает много ошибок в распознавании символов.
Исправляй только самые явные и понятные места. Если фрагмент текста сложно разобрать, то сохраняй его в том же виде, в котором и получил.
Сохраняй имена собственные и числительные как есть. Сохраняй исходную последовательность слов."""
    examples = "Примеры:\n"
    for i, (human_page, model_page) in enumerate(zip(human_pages, model_pages, strict=True)):
        for human_line, model_line in zip(human_page.split("\n"), model_page.split("\n"), strict=True):
            human_line = human_line.strip()
            model_line = model_line.strip()
            examples += f"{model_line} -> {human_line}\n"
    return prompt + "\n\n" + examples

In [11]:
system_prompt = make_system_prompt(human_pages_val, model_pages_val)
print(system_prompt)

NameError: name 'human_pages_val' is not defined

## Обрабатывем тестовые строки независимо

In [15]:
def get_response(question, access_token=tokens["access_token"], system_prompt=system_prompt, temperature=0, model="chatgpt-4o-latest", print_errors=True):
    response = json.loads(
        requests.post(
            'https://openai-proxy.tcsbank.ru/public/v1/chat/completions', 
            headers={
                "Authorization": f"Bearer {access_token}",
                "Content-Type": "application/json",
                "x-proxy-mask-critical-data": "1",
                "x-proxy-unmask-critical-data": "1",
            },
            json={
                "model": model,
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user",  "content": question}
                ],
                "temperature": temperature,
            },
        ).text
    )
    if "choices" not in response:
        if print_errors:
            print(response)
        return None
    return response["choices"][0]["message"]["content"]

In [16]:
print(get_response(model_pages_test[3].split("\n")[0] + " -> "))

Мартъ. 3-го. Уѣхалъ въ Петербургъ, ибо дѣло приняло дурной.


In [17]:
from tqdm import tqdm
import time


def get_correction(
    model_pages,
    **get_response_kwargs
):
    corrected_pages = []
    for model_page in tqdm(model_pages):
        corrected_lines = []
        for model_line in tqdm(model_page.split("\n"), leave=False):
            model_line = model_line.strip()
            corrected_line = get_response(model_line + " -> ", **get_response_kwargs)
            if corrected_line is None:
                time.sleep(10)
                corrected_line = get_response(model_line + " -> ", **get_response_kwargs)
                if corrected_line is None:
                    raise Exception(f"Could not get response for {model_line}")
            corrected_lines.append(corrected_line)
        corrected_pages.append("\n".join(corrected_lines))
    return corrected_pages

In [104]:
corrected_pages_test = get_correction(model_pages_test, print_errors=False)

100%|██████████| 5/5 [09:10<00:00, 110.12s/it]


In [105]:
for human_page, corrected_page in zip(human_pages_test, corrected_pages_test, strict=True):
    human_lines = human_page.strip().split("\n")
    corrected_lines = corrected_page.strip().split("\n")
    if len(human_lines) == len(corrected_lines):
        print("ok")
    else:
        print("not ok: ", end="")
        print(len(human_lines), len(corrected_lines))

ok
ok
ok
ok
ok


In [106]:
# отдельные строки как примеры
print(f"CER до GPT:    {calc_cer_from_pages(human_pages_test, model_pages_test, by_lines=False) : .4f}")
print(f"CER после GPT: {calc_cer_from_pages(human_pages_test, corrected_pages_test, by_lines=False) : .4f}")

CER до GPT:     0.2167
CER после GPT:  0.2150


In [112]:
# отдельные строки как примеры
print(f"WER до GPT:    {calc_wer_from_pages(human_pages_test, model_pages_test) : .4f}")
print(f"WER после GPT: {calc_wer_from_pages(human_pages_test, corrected_pages_test) : .4f}")

WER до GPT:     0.5991
WER после GPT:  0.5178


In [ ]:
# целые страницы как примеры
print(f"{calc_cer_from_pages(human_pages_test, model_pages_test, by_lines=False) : .4f}")
print(f"{calc_cer_from_pages(human_pages_test, corrected_pages_test, by_lines=False) : .4f}")

 0.2167
 0.2636


In [107]:
for model_page, corrected_page, human_page in zip(model_pages_test, corrected_pages_test, human_pages_test, strict=True):
    for model_line, corrected_line, human_line in zip(model_page.split("\n"), corrected_page.split("\n"), human_page.split("\n"), strict=True):
        print(f"Model:   {model_line}")
        print(f"ChatGPT: {corrected_line}")
        print(f"Human:   {human_line}")
        print()

Model:   Рябикова
ChatGPT: Рябикова
Human:   Рябчикова

Model:   Онь дсиля въ работникахъ и конца, и кроалъ что-т
ChatGPT: Онъ дѣлялъ въ работникахъ и конца, и края что-то.
Human:   Онъ де жилъ въ работникахъ у купца ; укралъ что-то

Model:   ителъ удосилъ его палкой-по Голавѣ отъ чеи у не..
ChatGPT: ителъ ударилъ его палкой по головѣ, отъ чего у не...
Human:   и тотъ ударилъ его палкою по головѣ, отчаго у него

Model:   и дѣлилаь удто бѣлая горянька. Обранясе къ Губер-
ChatGPT: и дѣлилась, будто бѣлая горлинка. Обращаясь къ Губер-
Human:   и сдѣлалась будто бѣлая горячка. Обратясь къ губер-

Model:   натору. В - вросодъ въ, А ріес le . Смумаюсь.
ChatGPT: натору. Въ – вопросъ въ А. Ріешѣніе. Сомневаюсь.
Human:   натору: В – f! Prens-le, pince-le! – Слушаюсь.

Model:   Болья никакилъ раторяженійи не было сцѣлано. Вороемъ
ChatGPT: Большаго никакихъ распоряженій не было сдѣлано. Воробьемъ
Human:   Болѣе никакихъ распоряженій и не было сдѣлано. Впрочемъ,

Model:   « гасовые возватлѣтсь на 

In [ ]:
# with open("corrected_pages_test.pkl", "wb") as f:
#     pickle.dump(corrected_pages_test, f)

In [ ]:
# import pickle


# with open("corrected_pages_test.pkl", "rb") as f:
#     loaded = pickle.load(f)

# loaded

['Рябикова\nОнъ дѣлялъ въ работникахъ и конца, и края что-то.\nителъ ударилъ его палкой по головѣ, отъ чего у не...\nи дѣлилась, будто бѣлая горлинка. Обращаясь къ Губер-\nнатору. Въ – вопросъ въ А. Ріешѣніе. Сомневаюсь.\nБольшаго никакихъ распоряженій не было сдѣлано. Воробьемъ\n«Гласные возвратились на прежнія мѣста – ибо\nвъ прочайшемъ случаѣ и острожные остались бы безъ.\nвсякаго кожуха.\nВечеромъ читалъ піэссу – ядѣ чрезвычайно\nпонравилась. – Беккеру также.\n10-го. Утромъ выѣхалъ въ Дѣдкаевское. Почивалъ в...',
 '1856. Годъ. Май.\n23е. 25е. 26е. Въ Воскресенскомъ хлѣбо-\n25-е. Пустилъ Заводъ. Вечеромъ пріѣхалъ Ѳедоръ изъ\nСтемной – тамъ въ сѣни подышается. Въ Воскресенскомъ\nСъ февраля поступилъ и управлялъ Зерунъ.\nДѣло идётъ довольно хорошо. Садку сдѣлалъ по\nпрачечной сторонѣ ларка отъ дома къ рѣкѣ. Около-\nдома проводить время. Ставили крестъ мнѣ Туле.\nбы отъ дома къ Копонину.\nИ рано утромъ выѣхалъ въ Москву. Послалъ за\nКавъ кимъ, который явился съ предложеніемъ отъ\nИвача

## При коррекции тестовых строк учитывеем историю

In [28]:
def get_response_v2(
    question,
    conversation_history=None,
    access_token=tokens["access_token"],
    system_prompt=system_prompt,
    temperature=0,
    model="chatgpt-4o-latest",
    print_errors=True
):
    # Если история разговора отсутствует, инициализируем ее с системным сообщением
    if conversation_history is None:
        conversation_history = [{"role": "system", "content": system_prompt}]
    # Добавляем вопрос пользователя в историю разговора
    conversation_history.append({"role": "user", "content": question})

    response = requests.post(
        'https://openai-proxy.tcsbank.ru/public/v1/chat/completions', 
        headers={
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json",
            "x-proxy-mask-critical-data": "1",
            "x-proxy-unmask-critical-data": "1",
        },
        json={
            "model": model,
            "messages": conversation_history,
            "temperature": temperature,
        },
    ).json()
    if "choices" not in response:
        if print_errors:
            print(response)
        return None, conversation_history[:-1]
    
    assistant_message = response["choices"][0]["message"]["content"]
    conversation_history.append({"role": "assistant", "content": assistant_message + "\n"})
    
    return assistant_message, conversation_history

In [38]:
from tqdm import tqdm
import time


def get_correction_v2(
    model_pages,
    sleep_seconds=10,
    n_tries=4,
    **get_response_kwargs
):
    corrected_pages = []
    conversation_history = None
    for model_page in tqdm(model_pages):
        corrected_lines = []
        for model_line in tqdm(model_page.split("\n"), leave=False):
            model_line = model_line.strip()

            for try_n in range(n_tries):
                corrected_line, conversation_history = get_response_v2(
                    model_line + " -> ", conversation_history, **get_response_kwargs
                )
                if corrected_line is not None:
                    break
                elif try_n == n_tries - 1:
                    # last try faliled
                    raise Exception(f"Could not get response for {model_line}")
                else:
                    time.sleep(sleep_seconds)

            corrected_lines.append(corrected_line)
        corrected_pages.append("\n".join(corrected_lines))
    return corrected_pages

In [54]:
c = get_correction_v2([model_pages_test[0]], print_errors=False)

100%|██████████| 1/1 [01:27<00:00, 87.44s/it]


In [56]:
print(c[0])

Рябикова
Онъ дѣлялъ въ работникахъ и конца, и края что-то
итель ударилъ его палкой по головѣ, отъ чего у не...
и дѣлилась, будто бѣлая горлинка. Обращаясь къ Губер-
натору. Въ вѣросходъ въ. А рѣшилъ. Смущаюсь.
Больше никакихъ распоряженій не было сдѣлано. Вороемъ
«Гласные возвратились на прежнія мѣста – ибо
въ прочемъ случаѣ и острожные остались бы безъ.
всякаго кожуха. 
.Вечеромъ читалъ піэссу – ядѣ чрезвычайно
понравилась. – Беккеру также.
10-го. Утромъ выѣхалъ въ Дѣдюхинское. Почувствовалъ в


In [57]:
corrected_pages_test = get_correction_v2(model_pages_test, print_errors=False)

100%|██████████| 5/5 [12:13<00:00, 146.67s/it]


In [58]:
print(corrected_pages_test[0])

Рябикова
Онъ дѣлялъ въ работникахъ и конца, и края что-то
итель ударилъ его палкой по головѣ, отъ чего у не...
и дѣлилась, будто бѣлая горлинка. Обращаясь къ Губер-
натору. Въ вопросѣ въ А... Смутился.
Больше никакихъ распоряженій не было сдѣлано. Вороемъ
«Газовые возвратились на прежнія мѣста – ибо
въ прочемъ случаѣ и острожные остались бы безъ.
всякаго кожуха.
Вечеромъ читалъ піэссу – ядѣ чрезвычайно
понравилась. – Беккеру также.
10-го. Утромъ выѣхалъ въ Дѣдюхинское. Почивалъ в


In [59]:
for human_page, corrected_page in zip(human_pages_test, corrected_pages_test, strict=True):
    human_lines = human_page.strip().split("\n")
    corrected_lines = corrected_page.strip().split("\n")
    if len(human_lines) == len(corrected_lines):
        print("ok")
    else:
        print("not ok: ", end="")
        print(len(human_lines), len(corrected_lines))

ok
ok
ok
ok
ok


In [60]:
# отдельные строки как примеры, учитываем историю
print(f"CER до GPT:    {calc_cer_from_pages(human_pages_test, model_pages_test, by_lines=False) : .4f}")
print(f"CER после GPT: {calc_cer_from_pages(human_pages_test, corrected_pages_test, by_lines=False) : .4f}")

CER до GPT:     0.2167
CER после GPT:  0.2062


In [61]:
# отдельные строки как примеры, учитываем историю
print(f"WER до GPT:    {calc_wer_from_pages(human_pages_test, model_pages_test) : .4f}")
print(f"WER после GPT: {calc_wer_from_pages(human_pages_test, corrected_pages_test) : .4f}")

WER до GPT:     0.5991
WER после GPT:  0.4879


In [62]:
for model_page, corrected_page, human_page in zip(model_pages_test, corrected_pages_test, human_pages_test, strict=True):
    for model_line, corrected_line, human_line in zip(model_page.split("\n"), corrected_page.split("\n"), human_page.split("\n"), strict=True):
        print(f"Model:   {model_line}")
        print(f"ChatGPT: {corrected_line}")
        print(f"Human:   {human_line}")
        print()

Model:   Рябикова
ChatGPT: Рябикова
Human:   Рябчикова

Model:   Онь дсиля въ работникахъ и конца, и кроалъ что-т
ChatGPT: Онъ дѣлялъ въ работникахъ и конца, и края что-то
Human:   Онъ де жилъ въ работникахъ у купца ; укралъ что-то

Model:   ителъ удосилъ его палкой-по Голавѣ отъ чеи у не..
ChatGPT: итель ударилъ его палкой по головѣ, отъ чего у не...
Human:   и тотъ ударилъ его палкою по головѣ, отчаго у него

Model:   и дѣлилаь удто бѣлая горянька. Обранясе къ Губер-
ChatGPT: и дѣлилась, будто бѣлая горлинка. Обращаясь къ Губер-
Human:   и сдѣлалась будто бѣлая горячка. Обратясь къ губер-

Model:   натору. В - вросодъ въ, А ріес le . Смумаюсь.
ChatGPT: натору. Въ вопросѣ въ А... Смутился.
Human:   натору: В – f! Prens-le, pince-le! – Слушаюсь.

Model:   Болья никакилъ раторяженійи не было сцѣлано. Вороемъ
ChatGPT: Больше никакихъ распоряженій не было сдѣлано. Вороемъ
Human:   Болѣе никакихъ распоряженій и не было сдѣлано. Впрочемъ,

Model:   « гасовые возватлѣтсь на преженія мѣста – 

## Коррекция после фикса с паддингом

In [11]:
eval_results_path = Path("/Users/v.p.zykov/Downloads/telegram")

with open(eval_results_path / "eval_train.pkl", "rb") as f:
    eval_train = pickle.load(f)

with open(eval_results_path / "eval_test.pkl", "rb") as f:
    eval_test = pickle.load(f)

with open(eval_results_path / "eval_val.pkl", "rb") as f:
    eval_val = pickle.load(f)

In [12]:
eval_test.keys()

dict_keys(['str_preds', 'str_gt', 'cer', 'wer', 'cers', 'wers'])

In [13]:
import sys
sys.path.append('..')

from basic.utils import edit_cer_from_list, nb_chars_from_list, edit_wer_from_list, nb_words_from_list

In [14]:
def calc_cer_from_list(gt: list[str], pred: list[str]):
    return edit_cer_from_list(gt, pred) / nb_chars_from_list(gt)

def calc_wer_from_list(gt: list[str], pred: list[str]):
    return edit_wer_from_list(gt, pred) / nb_words_from_list(gt)

In [15]:
assert calc_cer_from_list(eval_test["str_gt"], eval_test["str_preds"]) == eval_test["cer"]

In [16]:
assert calc_wer_from_list(eval_test["str_gt"], eval_test["str_preds"]) == eval_test["wer"]

In [44]:
def make_system_prompt_v2(val_gt: list[str], val_preds: list[str], train_gt: list[str], train_preds: list[str], limit_train_lines=250):
    prompt = f"""Твоя задача - корректировать входной текст, исправляя в нем ошибки.
Это текст, распознанный моделью компьютерного зрения с рукописей. Рукописи написаны на русском языке 19 века (также иногда присутствуют другие языки).
Цель - получить максимально близкий к рукописи вариант расшифровки.
Модель допускает ошибки в распознавании символов.
Исправляй только самые явные и понятные места. Если фрагмент текста сложно разобрать, то сохраняй его в том же виде, в котором и получил.
Сохраняй имена собственные и числительные как есть. Сохраняй исходную последовательность слов.
Тебе будут подаваться строки, расспознанные моделью компьютерного зрения, к которым в конце добавлены символы " -> ". Тебе нужно вернуть исправленную строку."""
    # examples = "Правильный текст из обучающей выборки:\n"
    # for i, (human_line, model_line) in enumerate(zip(train_gt, train_preds, strict=True)):
    #     examples += f"{human_line}\n"
    #     if i + 1 >= limit_train_lines:
    #         break
    # examples += "\n\nПримеры исправлений, требующихся от тебя:\n"
    # for i, (human_line, model_line) in enumerate(zip(val_gt, val_preds, strict=True)):
    #     examples += f"{model_line} -> {human_line}\n"
    train_gt = train_gt[:limit_train_lines]
    train_preds = train_preds[:limit_train_lines]
    gt = train_gt + val_gt
    preds = train_preds + val_preds
    examples = "Примеры:\n"
    for i, (human_line, model_line) in enumerate(zip(gt, preds, strict=True)):
        examples += f"{model_line} -> {human_line}\n"
    return prompt + "\n\n" + examples

In [45]:
len(eval_train["str_preds"])

1599

In [46]:
system_prompt = make_system_prompt_v2(eval_val["str_gt"], eval_val["str_preds"], eval_train["str_gt"], eval_train["str_preds"])
print(system_prompt)

Твоя задача - корректировать входной текст, исправляя в нем ошибки.
Это текст, распознанный моделью компьютерного зрения с рукописей. Рукописи написаны на русском языке 19 века (также иногда присутствуют другие языки).
Цель - получить максимально близкий к рукописи вариант расшифровки.
Модель допускает ошибки в распознавании символов.
Исправляй только самые явные и понятные места. Если фрагмент текста сложно разобрать, то сохраняй его в том же виде, в котором и получил.
Сохраняй имена собственные и числительные как есть. Сохраняй исходную последовательность слов.
Тебе будут подаваться строки, расспознанные моделью компьютерного зрения, к которым в конце добавлены символы " -> ". Тебе нужно вернуть исправленную строку.

Примеры:
вечеръ. Всталъ въ 6-м часу. -> вечеръ. Всталъ въ 6мъ часу.
21е. Перемѣна погоды. Довдикъ. Свѣжо. Всталъ -> 21е. Перемѣна погоды. Дождикъ, свѣжо, всталъ
въ 6-тъ часу. Купался къ дождь. Гредъ очень мало. Чув- -> въ 6мъ часу. Купался в дождь. Гребъ очень мало. Чув-

In [50]:
from tqdm import tqdm
import time


def get_correction_v3(
    model_lines,
    sleep_seconds=10,
    n_tries=4,
    print_correction=False,
    **get_response_kwargs
):
    corrected_lines = []
    conversation_history = None
    for model_line in tqdm(model_lines):
        for try_n in range(n_tries):
            corrected_line, conversation_history = get_response_v2(
                model_line + " -> ", conversation_history, **get_response_kwargs
            )
            if corrected_line is not None:
                corrected_lines.append(corrected_line)
                if print_correction:
                    print(model_line + " -> " + corrected_line)
                break
            elif try_n == n_tries - 1:
                # last try faliled
                raise Exception(f"Could not get response for {model_line}")
            else:
                time.sleep(sleep_seconds)

    return corrected_lines

In [53]:
corrected_test_lines = get_correction_v3(eval_test["str_preds"], system_prompt=system_prompt, print_errors=True, print_correction=True)

  1%|▏         | 1/74 [00:09<11:06,  9.12s/it]

Рябикова -> Рябикова


  3%|▎         | 2/74 [00:22<13:49, 11.52s/it]

Онъ де жилъ въ раборникахъ, у купца, у кралъ, что-т -> Онъ, дескать, жилъ въ работникахъ, у купца, укралъ что-то.


  4%|▍         | 3/74 [00:37<15:28, 13.08s/it]

...

крестьянами. Какъ замѣшно это было Сильной странно -> крестьянами. Какъ замѣшано это было – сильно странно.
{'error': {'code': 8, 'message': "openaiproxy.(*ServiceImplementation).processChatCompletionRequest: can't do chat completion - proxy.(*UseCase).ChatCompletion: can't check request - code: <nil>, message: proxy: token limit reached. Limit: 50000 / min. Available: 0 / min. Retry after: 18.627260953s. Reset after: 50.54726094s., param: <nil>, type: proxy_limit_reached", 'details': [{'@type': 'type.googleapis.com/openaiproxy.v1.ErrorResponse', 'error': {'code': '', 'message': 'proxy: token limit reached. Limit: 50000 / min. Available: 0 / min. Retry after: 18.627260953s. Reset after: 50.54726094s.', 'param': '', 'type': 'proxy_limit_reached', 'critical_data_details': [], 'limit_reached_details': {'type': 'token', 'retry_after': 18.627260953, 'reset_after': 50.54726094}}}, {'@type': 'type.googleapis.com/openaiproxy.v1.StatusCode', 'value': 429}]}}
{'error': {'code': 8, 'message': "op

 99%|█████████▊| 73/74 [29:59<00:26, 26.37s/it]

онъ довста Крестьямъ до большаго благю состоянія и богат- -> онъ довёлъ крестьянъ до большого благосостоянія и богат-
{'error': {'code': 8, 'message': "openaiproxy.(*ServiceImplementation).processChatCompletionRequest: can't do chat completion - proxy.(*UseCase).ChatCompletion: can't check request - code: <nil>, message: proxy: token limit reached. Limit: 50000 / min. Available: 0 / min. Retry after: 24.084764957s. Reset after: 55.927964955s., param: <nil>, type: proxy_limit_reached", 'details': [{'@type': 'type.googleapis.com/openaiproxy.v1.ErrorResponse', 'error': {'code': '', 'message': 'proxy: token limit reached. Limit: 50000 / min. Available: 0 / min. Retry after: 24.084764957s. Reset after: 55.927964955s.', 'param': '', 'type': 'proxy_limit_reached', 'critical_data_details': [], 'limit_reached_details': {'type': 'token', 'retry_after': 24.084764957, 'reset_after': 55.927964955}}}, {'@type': 'type.googleapis.com/openaiproxy.v1.StatusCode', 'value': 429}]}}
{'error': {'code': 8, '

100%|██████████| 74/74 [30:33<00:00, 24.77s/it]

ства, получилъ имѣніе разоренное и кресетьясъ протившотя – -> ства, получилъ имѣніе разоренное, и крестьяне восстали противъ него –


In [54]:
# отдельные строки как примеры, учитываем историю, после паддинга + учитываем train (pred -> gt)

print(f"CER до GPT:    {eval_test["cer"]: .6f}")
print(f"CER после GPT: {calc_cer_from_list(eval_test["str_gt"], corrected_test_lines) : .6f}")
print(f"WER до GPT:    {eval_test["wer"]: .6f}")
print(f"WER после GPT: {calc_wer_from_list(eval_test["str_gt"], corrected_test_lines) : .6f}")
print()

for model_line, corrected_line, human_line in zip(eval_test["str_preds"], corrected_test_lines, eval_test["str_gt"], strict=True):
    model_cer = editdistance.eval(model_line, human_line) / len(human_line)
    gpt_cer = editdistance.eval(corrected_line, human_line) / len(human_line)
    print(f"Model:   {model_line} (cer = {model_cer * 100:.2f}%)")
    print(f"ChatGPT: {corrected_line} (cer = {gpt_cer * 100:.2f}%)")
    print(f"Human:   {human_line}")
    print()

CER до GPT:     0.159323
CER после GPT:  0.158739
WER до GPT:     0.506419
WER после GPT:  0.383738

Model:   Рябикова (cer = 11.11%)
ChatGPT: Рябикова (cer = 11.11%)
Human:   Рябчикова

Model:   Онъ де жилъ въ раборникахъ, у купца, у кралъ, что-т (cer = 14.00%)
ChatGPT: Онъ, дескать, жилъ въ работникахъ, у купца, укралъ что-то. (cer = 22.00%)
Human:   Онъ де жилъ въ работникахъ у купца ; укралъ что-то

Model:   и тотъ удорилъ его палькою по головѣ отъ чего у нем (cer = 16.00%)
ChatGPT: и тотъ ударилъ его палкою по головѣ, отъ чего у него (cer = 6.00%)
Human:   и тотъ ударилъ его палкою по головѣ, отчаго у него

Model:   и сдѣлалась рудето бѣлая горяька. Обратясеь къ Губер- (cer = 9.80%)
ChatGPT: и сдѣлалась, вроде, бѣлая горячка. Обратился къ Губер- (cer = 19.61%)
Human:   и сдѣлалась будто бѣлая горячка. Обратясь къ губер-

Model:   наторуг. В- р росодъ въ, t ріscсe l.1. cлумаюсь. (cer = 58.70%)
ChatGPT: Губернатору. Въ разсудокъ въ Tрясце... Сомневаюсь. (cer = 76.09%)
Human:   натор

In [ ]:
# отдельные строки как примеры, учитываем историю, после паддинга + учитываем train (gt)

# print(f"CER до GPT:    {eval_test["cer"]: .6f}")
# print(f"CER после GPT: {calc_cer_from_list(eval_test["str_gt"], corrected_test_lines) : .6f}")
# print(f"WER до GPT:    {eval_test["wer"]: .6f}")
# print(f"WER после GPT: {calc_wer_from_list(eval_test["str_gt"], corrected_test_lines) : .6f}")
# print()

# for model_line, corrected_line, human_line in zip(eval_test["str_preds"], corrected_test_lines, eval_test["str_gt"], strict=True):
#    model_cer = editdistance.eval(model_line, human_line) / len(human_line)
#    gpt_cer = editdistance.eval(corrected_line, human_line) / len(human_line)
#    print(f"Model:   {model_line} (cer = {model_cer * 100:.2f}%)")
#    print(f"ChatGPT: {corrected_line} (cer = {gpt_cer * 100:.2f}%)")
#    print(f"Human:   {human_line}")
#    print()

CER до GPT:     0.159323
CER после GPT:  0.156405
WER до GPT:     0.506419
WER после GPT:  0.366619

Model:   Рябикова (cer = 11.11%)
ChatGPT: Рябикова -> Рябиникова (cer = 155.56%)
Human:   Рябчикова

Model:   Онъ де жилъ въ раборникахъ, у купца, у кралъ, что-т (cer = 14.00%)
ChatGPT: Онъ, дескать, жилъ въ работникахъ, у купца, укралъ что-то. (cer = 22.00%)
Human:   Онъ де жилъ въ работникахъ у купца ; укралъ что-то

Model:   и тотъ удорилъ его палькою по головѣ отъ чего у нем (cer = 16.00%)
ChatGPT: и тотъ ударилъ его палкою по головѣ, отъ чего у него (cer = 6.00%)
Human:   и тотъ ударилъ его палкою по головѣ, отчаго у него

Model:   и сдѣлалась рудето бѣлая горяька. Обратясеь къ Губер- (cer = 9.80%)
ChatGPT: и сдѣлалась, говорятъ, бѣлая горячка. Обратилсяъ къ Губер- (cer = 27.45%)
Human:   и сдѣлалась будто бѣлая горячка. Обратясь къ губер-

Model:   наторуг. В- р росодъ въ, t ріscсe l.1. cлумаюсь. (cer = 58.70%)
ChatGPT: губернатору. Въ разговорѣ въ тюрьмѣ, сломался. (cer = 78.26%)

In [ ]:
# отдельные строки как примеры, учитываем историю, после паддинга
# print(f"CER до GPT:    {eval_test["cer"]: .6f}")
# print(f"CER после GPT: {calc_cer_from_list(eval_test["str_gt"], corrected_test_lines) : .6f}")

CER до GPT:     0.159323
CER после GPT:  0.165159


In [ ]:
# отдельные строки как примеры, учитываем историю, после паддинга
# print(f"WER до GPT:    {eval_test["wer"]: .6f}")
# print(f"WER после GPT: {calc_wer_from_list(eval_test["str_gt"], corrected_test_lines) : .6f}")

WER до GPT:     0.506419
WER после GPT:  0.425107


In [ ]:
# for model_line, corrected_line, human_line in zip(eval_test["str_preds"], corrected_test_lines, eval_test["str_gt"], strict=True):
#     model_cer = editdistance.eval(model_line, human_line) / len(human_line)
#     gpt_cer = editdistance.eval(corrected_line, human_line) / len(human_line)
#     print(f"Model:   {model_line} (cer = {model_cer * 100:.2f}%)")
#     print(f"ChatGPT: {corrected_line} (cer = {gpt_cer * 100:.2f}%)")
#     print(f"Human:   {human_line}")
#     print()

Model:   Рябикова (cer = 11.11%)
ChatGPT: Рябикова (cer = 11.11%)
Human:   Рябчикова

Model:   Онъ де жилъ въ раборникахъ, у купца, у кралъ, что-т (cer = 14.00%)
ChatGPT: Онъ, де, жилъ въ Рабочникахъ, у купца, укралъ что-то. (cer = 16.00%)
Human:   Онъ де жилъ въ работникахъ у купца ; укралъ что-то

Model:   и тотъ удорилъ его палькою по головѣ отъ чего у нем (cer = 16.00%)
ChatGPT: и тотъ ударилъ его палкою по головѣ, отъ чего у нѣм- (cer = 12.00%)
Human:   и тотъ ударилъ его палкою по головѣ, отчаго у него

Model:   и сдѣлалась рудето бѣлая горяька. Обратясеь къ Губер- (cer = 9.80%)
ChatGPT: и сдѣлалась, вроде, бѣлая горячка. Обратился къ Губер- (cer = 19.61%)
Human:   и сдѣлалась будто бѣлая горячка. Обратясь къ губер-

Model:   наторуг. В- р росодъ въ, t ріscсe l.1. cлумаюсь. (cer = 58.70%)
ChatGPT: Губернатору. Въ разсудъ въ Трясце... Сомневаюсь. (cer = 76.09%)
Human:   натору: В – f! Prens-le, pince-le! – Слушаюсь.

Model:   Болѣе никакихъ разпоряженій и не было сцѣлано.– Вороемъ

## DeepSeek

In [112]:
def make_system_prompt_v3(val_gt: list[str], val_preds: list[str], train_gt: list[str], train_preds: list[str], limit_train_lines=250):
    prompt = f"""Твоя задача - корректировать входной текст, исправляя в нем ошибки.
Это текст, распознанный моделью компьютерного зрения с рукописей.
Рукописи написаны на русском языке 19 века (также иногда присутствуют другие языки).
Цель - получить максимально близкий к рукописи вариант расшифровки.
Модель компьютерного зрения допускает ошибки в распознавании символов.
Модель распознает текст по отдельным строкам. Ты будешь участвовать в этом пайплайне, корректируя расшифровку каждой строки.
Исправляй только самые явные и понятные места. Если фрагмент текста сложно разобрать, то сохраняй его в том же виде, в котором и получил.
Сохраняй имена собственные и числительные как есть. Сохраняй исходную последовательность слов.
Помни, что ты работаешь только с отдельными строками. К поданной текущей строке нужно вернуть исправленную только для этой строки расшифровку.
Давай только ответ без всего лишнего."""
    # examples = "Правильный текст из обучающей выборки:\n"
    # for i, (human_line, model_line) in enumerate(zip(train_gt, train_preds, strict=True)):
    #     examples += f"{human_line}\n"
    #     if i + 1 >= limit_train_lines:
    #         break
    # examples += "\n\nПримеры исправлений, требующихся от тебя:\n"
    # for i, (human_line, model_line) in enumerate(zip(val_gt, val_preds, strict=True)):
    #     examples += f"{model_line} -> {human_line}\n"
    train_gt = train_gt[:limit_train_lines]
    train_preds = train_preds[:limit_train_lines]
    gt = train_gt + val_gt
    preds = train_preds + val_preds
    examples = 'Примеры (в каждой строке ниже содержится по одному примеру правильной работы: до " -> " указана распознанная строка, а после - желаемый ответ):\n'
    for i, (human_line, model_line) in enumerate(zip(gt, preds, strict=True)):
        examples += f"{model_line} -> {human_line}\n"
    return prompt + "\n\n" + examples

In [ ]:
system_prompt = make_system_prompt_v3(eval_val["str_gt"], eval_val["str_preds"], eval_train["str_gt"], eval_train["str_preds"], limit_train_lines=0)
print(system_prompt)

Твоя задача - корректировать входной текст, исправляя в нем ошибки.
Это текст, распознанный моделью компьютерного зрения с рукописей.
Рукописи написаны на русском языке 19 века (также иногда присутствуют другие языки).
Цель - получить максимально близкий к рукописи вариант расшифровки.
Модель компьютерного зрения допускает ошибки в распознавании символов.
Модель распознает текст по отдельным строкам. Ты будешь участвовать в этом пайплайне, корректируя расшифровку каждой строки.
Исправляй только самые явные и понятные места. Если фрагмент текста сложно разобрать, то сохраняй его в том же виде, в котором и получил.
Сохраняй имена собственные и числительные как есть. Сохраняй исходную последовательность слов.
Помни, что ты работаешь только с отдельными строками. К поданной текущей строке нужно вернуть исправленную только для этой строки расшифровку.
Давай только ответ без всего лишнего.

Примеры (в каждой строке ниже содержится по одному примеру правильной работы: до " -> " указана распоз

In [115]:
corrected_test_lines = get_correction_v3(eval_test["str_preds"][:1] + eval_test["str_preds"], system_prompt=system_prompt, print_errors=True, print_correction=True, model="DeepSeek-R1", n_tries=6)

  1%|▏         | 1/75 [00:20<24:51, 20.15s/it]

Рябикова -> <think>
Хорошо, давайте разберёмся с этой строкой "лагаРябикова". Нужно понять, какие ошибки здесь могли возникнуть при распознавании рукописного текста. 

Первое, что бросается в глаза, это начало слова "лага". Возможно, это опечатка или неправильно распознанный символ. В русском языке нет слова "лага", которое бы подходило по контексту. Скорее всего, это часть фамилии или имени собственного. 

Далее идёт "Рябикова". Фамилия "Рябиков" существует, и с суффиксом "-а" это может быть женская форма. Но сочетание "лагаРябикова" выглядит странно. Возможно, здесь пропущен пробел или дефис. Например, если это "Лага Рябикова" или "Ла-Га Рябикова", но такие варианты маловероятны.

Ещё один вариант: первая часть слова "лага" может быть искажённым написанием другой буквы. Например, "лага" может быть "Лада" или "Лага" (если это имя или название). Но без контекста сложно сказать точно.

Также стоит проверить, не является ли это частью более длинного слова. Например, "лагаРябикова" может 

  3%|▎         | 2/75 [00:31<18:27, 15.17s/it]

Рябикова -> <think>
Хорошо, давайте разберемся с этим запросом. Пользователь предоставил примеры исправлений текста, распознанного с рукописей 19 века, и просит исправить строку "лагаРябикова". В примере он сам предположил, что "лага" может быть сокращением или частью имени, и предложил разделить слитное написание на два слова: "Лага Рябикова". 

Первым делом, нужно понять, что именно здесь не так. В исходной строке "лагаРябикова" слова слиты вместе. В русском языке имена собственные, особенно фамилии, обычно пишутся раздельно, если это не составная фамилия. Однако "лага" не является стандартной частью русских фамилий. Возможно, это опечатка или ошибка распознавания.

Судя по примерам, которые привел пользователь, в подобных случаях нужно разделять слитно написанные слова, особенно если это имена собственные. Например, в одном из примеров "бексеромъ" было исправлено на "Беккеромъ", что показывает, что важно сохранять правильное написание фамилий. 

Также в примере с "лагаРябикова" поль

  4%|▍         | 3/75 [00:45<17:31, 14.60s/it]

...

крестьянами. Какъ замѣшно это было Сильной странно -> 

Замѣчательный разсказъ о помѣщикѣ Малиновскомъ, убитомъ его  
крестьянами. Какъ замѣтно это было. Сильно странно
</think>

1854 годъ. Новый годъ на желѣзной дорогѣ. Былъ въ оскре-  
снискомъ. 15 числа выѣхалъ на Выксу. Амнtt Гол.  
Піэсса читалась съ успѣхомъ. Въ февралѣ  
долженъ былъ ѣхать въ Москву по поводу начавшагося  
слѣдствія. Ѣздилъ на нѣсколько дней въ Петербургъ.  
Истратилъ охотничьихъ вещей, чтобы охотиться.  
2-го Марта на первой недѣли воротился въ Москву.  
для пріема Дамъ. Онѣ пріѣхали поздно. Обѣдали въ 6-мъ часу – было  
довольно весело. – я опять вечеромъ игралъ въ карты.  
нѣтъ. Я отправилъ семью съ. Въ Тулу Запродать будущаго сахара.  
28-е. Всталъ рано, сполярничалъ. Распимывалъ и отправилъ Выю-  
сумскихъ 18 человѣкъ. На обратныхъ подводахъ затребовалъ изъ Москвы  
Либли, картины, бронзу. Если поживу еще, буду жить въ Россіи – такъ  
это здѣсь – въ кобыликѣ. –  
Замѣчательный разсказъ о помѣщикѣ Малиновско

 99%|█████████▊| 74/75 [26:56<00:36, 36.55s/it]

онъ довста Крестьямъ до большаго благю состоянія и богат- -> 

Замѣчательный разсказъ о помѣщикѣ Малиновскомъ, убитомъ его  
крестьянами. Какъ замѣтно это было. Сильно странно  
онъ довелъ крестьянъ до большаго благо состоянія и богат-
</think>

28-е. Всталъ рано, сполярничалъ. Распимывалъ и отправилъ Выю-  
сумскихъ 18 человѣкъ. На обратныхъ подводахъ затребовалъ изъ Москвы  
Либли, картины, бронзу. Если поживу еще, буду жить въ Россіи – такъ  
это здѣсь – въ кобыликѣ. –  
Замѣчательный разсказъ о помѣщикѣ Малиновскомъ, убитомъ его  
крестьянами. Какъ замѣтно это было. Сильно странно  
онъ довелъ крестьянъ до большаго благо состоянія и богат-
{'error': {'code': 8, 'message': "openaiproxy.(*ServiceImplementation).processChatCompletionRequest: can't do chat completion - proxy.(*UseCase).ChatCompletion: can't check request - code: <nil>, message: proxy: token limit reached. Limit: 50000 / min. Available: 0 / min. Retry after: 22.528001964s. Reset after: 45.288401961s., param: <nil>, type

100%|██████████| 75/75 [27:36<00:00, 22.08s/it]

ства, получилъ имѣніе разоренное и кресетьясъ протившотя – -> 

28-е. Всталъ рано, сполярничалъ. Распимывалъ и отправилъ Выю-  
сумскихъ 18 человѣкъ. На обратныхъ подводахъ затребовалъ изъ Москвы  
Либли, картины, бронзу. Если поживу еще, буду жить въ Россіи – такъ  
это здѣсь – въ кобыликѣ. –  
Замѣчательный разсказъ о помѣщикѣ Малиновскомъ, убитомъ его  
крестьянами. Какъ замѣтно это было. Сильно странно  
онъ довелъ крестьянъ до большаго благо состоянія и богат-  
ства, получилъ имѣніе разоренное и крестьянъ противъ себя –
</think>

28-е. Всталъ рано, сполярничалъ. Распимывалъ и отправилъ Выю-  
сумскихъ 18 человѣкъ. На обратныхъ подводахъ затребовалъ изъ Москвы  
Либли, картины, бронзу. Если поживу еще, буду жить въ Россіи – такъ  
это здѣсь – въ кобыликѣ. –  
Замѣчательный разсказъ о помѣщикѣ Малиновскомъ, убитомъ его  
крестьянами. Какъ замѣтно это было. Сильно странно  
онъ довелъ крестьянъ до большаго благо состоянія и богат-  
ства, получилъ имѣніе разоренное и крестьянъ проти

In [139]:
for i in range(1, len(corrected_test_lines)):
    print(corrected_test_lines[i])
    print("-" * 100)
    print()

<think>
Хорошо, давайте разберемся с этим запросом. Пользователь предоставил примеры исправлений текста, распознанного с рукописей 19 века, и просит исправить строку "лагаРябикова". В примере он сам предположил, что "лага" может быть сокращением или частью имени, и предложил разделить слитное написание на два слова: "Лага Рябикова". 

Первым делом, нужно понять, что именно здесь не так. В исходной строке "лагаРябикова" слова слиты вместе. В русском языке имена собственные, особенно фамилии, обычно пишутся раздельно, если это не составная фамилия. Однако "лага" не является стандартной частью русских фамилий. Возможно, это опечатка или ошибка распознавания.

Судя по примерам, которые привел пользователь, в подобных случаях нужно разделять слитно написанные слова, особенно если это имена собственные. Например, в одном из примеров "бексеромъ" было исправлено на "Беккеромъ", что показывает, что важно сохранять правильное написание фамилий. 

Также в примере с "лагаРябикова" пользователь пре

In [128]:
corrected_test_lines_ds = []
for i in range(1, len(corrected_test_lines)):
    line = corrected_test_lines[i]
    line = line[line.rfind("</think>") + len("</think>\n\n"):]
    line = line[line.rfind("\n") + 1:]
    corrected_test_lines_ds.append(line)

In [129]:
# отдельные строки как примеры, учитываем историю, после паддинга + учитываем train (pred -> gt)

print(f"CER до DS:    {eval_test["cer"]: .6f}")
print(f"CER после DS: {calc_cer_from_list(eval_test["str_gt"], corrected_test_lines_ds) : .6f}")
print(f"WER до DS:    {eval_test["wer"]: .6f}")
print(f"WER после DS: {calc_wer_from_list(eval_test["str_gt"], corrected_test_lines_ds) : .6f}")
print()

for model_line, corrected_line, human_line in zip(eval_test["str_preds"], corrected_test_lines_ds, eval_test["str_gt"], strict=True):
    model_cer = editdistance.eval(model_line, human_line) / len(human_line)
    gpt_cer = editdistance.eval(corrected_line, human_line) / len(human_line)
    print(f"Model:    {model_line} (cer = {model_cer * 100:.2f}%)")
    print(f"DeepSeek: {corrected_line} (cer = {gpt_cer * 100:.2f}%)")
    print(f"Human:    {human_line}")
    print()

CER до DS:     0.159323
CER после DS:  0.173913
WER до DS:     0.506419
WER после DS:  0.420827

Model:    Рябикова (cer = 11.11%)
DeepSeek: лагаРябикова -> Лага Рябикова (cer = 233.33%)
Human:    Рябчикова

Model:    Онъ де жилъ въ раборникахъ, у купца, у кралъ, что-т (cer = 14.00%)
DeepSeek: Онъ де жилъ въ рабочихъ, у купца, у краля, что-то. (cer = 22.00%)
Human:    Онъ де жилъ въ работникахъ у купца ; укралъ что-то

Model:    и тотъ удорилъ его палькою по головѣ отъ чего у нем (cer = 16.00%)
DeepSeek: и тотъ ударилъ его палкою по головѣ, отъ чего у него (cer = 6.00%)
Human:    и тотъ ударилъ его палкою по головѣ, отчаго у него

Model:    и сдѣлалась рудето бѣлая горяька. Обратясеь къ Губер- (cer = 9.80%)
DeepSeek: и сдѣлалась рудето бѣлая горячка. Обратясь къ Губер- (cer = 5.88%)
Human:    и сдѣлалась будто бѣлая горячка. Обратясь къ губер-

Model:    наторуг. В- р росодъ въ, t ріscсe l.1. cлумаюсь. (cer = 58.70%)
DeepSeek: наторуг. В- р росодъ въ, t ріscсe l.1. cлумаюсь. (cer = 58.